<a href="https://colab.research.google.com/github/bernatsort/Customer_Churn_Prevention/blob/main/customer_churn_prevention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Bernat Sort Rufat

MD004_Advanced data analysis and visualization tools

Proyecto Final de la asignatura Herramientas avanzadas de análisis y visualización de datos

Máster en Data Science

# Customer Churn Prevention



## Objetivos:

El objetivo de este proyecto es encontrar acciones concretas que nos ayuden a prevenir que un cliente haga churn.
Para ello, se seguirán una serie de pasos que nos ayudarán a cumplir el objetivo. 

- Análisis exploratorio de los datos:
    - Estadística descriptiva y calidad general de los datos.
    - Visualizaciones que ayuden a entender la distribución de las variables y categorías.
    - Visualizaciones que ayuden a entender la relación entre los atributos y la variable objetivo is_churned.
    - Resumen de las conclusiones sacadas en este punto.
    
    
- Análisis de datos mediante el uso de técnicas estadísticas:
    - De entre las siguientes pruebas (Correlación, PCA, ANOVA, Información Mútua) se seleccionen al menos 2 pruebas sobre las que basar el análisis. Para estas pruebas se pide:
        - Desarrollo teórico de la técnica: ¿Qué es?, ¿Para que sirve? ¿Por qué utilizamos éstas pruebas y no otras?
        - Desarrollo de la técnica sobre el conjunto de datos: Analizar y explicar los resultados obtenidos.
        - Podéis (y recomendamos) utilizar todas las ténicas enumeradas. El requisito del desarrollo teórico de la ténica a desarrollar se aplica únicamente a 2 de ellas como mínimo.


- Desarrollo del modelo de regresión logística: 
    - Desarrollo y justificación de las variables empleadas en el modelo.
    - Selección y justificación de la métrica de optimización del modelo.
    - Desarrollo de al menos 3 modelo y comparación de resultados (Matriz de confusión).

- Conclusiones y vías abiertas: 
    - ¿Qué recomendaciones le daríais a esta empresa para reducir churn? 
    - ¿Cómo os ayuda el modelo que habéis calculado a llegar a estas conclusiones?


## Carga de packages

In [ ]:
sessionInfo()

In [ ]:
install.packages('gridExtra')
install.packages('GGally')
install.packages('caret')
install.packages('naniar')
install.packages('mice')
install.packages('ltm')
install.packages('FSelectorRcpp')
install.packages('nortest')
install.packages('regclass')
install.packages("forcats")
install.packages("dplyr")

In [ ]:
library(ggplot2)

# install.packages('gridExtra')
library(gridExtra)

# install.packages ('GGally')
library(GGally)

library(caret)

library(dplyr)

library(RColorBrewer)

# install.packages('naniar')
library(naniar)

# install.packages('mice')
library(mice)

# install.packages('ltm')
library(ltm)

# install.packages('FSelectorRcpp')
library(FSelectorRcpp)


# install.packages('nortest')
library(nortest)

# install.packages('regclass')
library(regclass)

library(forcats)

## Carga de datos

In [ ]:
# cargamos los datos en el df
cust_churn_df <- read.csv(file="customer_churn_data.csv", header=TRUE, sep=",", dec=".")

## Exploratory Data Analysis

El csv customer_churn_data incluye datos cualitativos y cuantitativos de clientes de una empresa de telecomunicaciones india en la que se detallan aspectos de los clientes de la empresa. Está compuesto por 104143 instancias y 18 variables.

El dataset se compone de las siguientes columnas:

Descripción de las variables:
- device user’s – device brand (Categorical)
- first_payment_amount  – user’s first payment amount(Numeric)
- age  – user’s age(Numeric or categorical?)
- city  – user’s city(Categorical)
- number_of_cards  – #of cards user owns
- payments_initiated  – #of payments initiated by user
- payments_failed  – #of payments failed
- payments_completed – #of payments completed
- payments_completed_amount_first_7days  – amt of payment completed in first 7 days of joining
- reward_purchase_count_first_7days – #of rewards claimed in first 7 days
- coins_redeemed_first_7days – coins redeemed in first 7 days
- is_referral – is user a referred user
- visits_feature_1  – #of visits made by user to product feature 1
- visits_feature_2 – #of visits made by user to product feature 2
- given_permission_1 – has user given permission 1
- given_permission_2 – has user given permission 2
- user_id – user identifier
- is_churned – whether user churned

In [ ]:
head(cust_churn_df) # de forma predefinida retorna las primeras 6 observations
tail(cust_churn_df) # de forma predefinida retorna las últimas 6 observations

In [ ]:
# dimensión del df (filas y columnas)
dim(cust_churn_df)

- El dataset incluye 104143 instancias (filas) y 18 variables (columnas).

In [ ]:
# estructura interna compacta del df
str(cust_churn_df)

- Observamos que is_referral, given_permission_1, given_permission_2 y is_churned en realidad son variables categóricas. Por lo tanto, las convertiremos de numéricas a categóricas. 

- Logi es la forma abreviada de logical, que es un tipo de datos en R que representa un valor booleano de TRUE o FALSE. En este caso, is_referral era originalmente una variable lógica con valores TRUE y FALSE.

- Sería una buena idea convertir la variable is_referral en un factor con 0 y 1, ya es una variable binaria. Esto facilitará la interpretación de la variable y su uso en la modelización.
    - 0: FALSE (user is not a referred user)
    - 1: TRUE (user is a referred user)

In [ ]:
cust_churn_df$is_referral <- as.factor(ifelse(cust_churn_df$is_referral, 1, 0))
cust_churn_df$is_referral

In [ ]:
# convertimos given_permission_1, given_permission_2, is_churned a categóricas (factor)
cust_churn_df$given_permission_1 = as.factor(cust_churn_df$given_permission_1)
cust_churn_df$given_permission_2 = as.factor(cust_churn_df$given_permission_2)
cust_churn_df$is_churned = as.factor(cust_churn_df$is_churned)

In [ ]:
str(cust_churn_df)

In [ ]:
# visión general del df
summary(cust_churn_df)

La función summary() proporciona algunas estadísticas básicas para cada variable del conjunto de datos.
- Podemos observar que algunas variables tienen NA values. 
- Podemos ver que variables como coins_redeemed_first_7days, first_payment_amount y payments_completed_amount_first_7days tienen un amplio rango de valores y presentan una gran diferencia entre la mediana y la media.



In [ ]:
# miramos si hay valores nulos
sum(is.na(cust_churn_df))

- Observamos que hay un total de 31530 valores nulos en el dataset. 

Mostramos las columnas con NA values y el número de NA values:
- Creamos el vector na_cols que contiene los nombres de las columnas de cust_churn_df que tienen al menos un NA value. 
- La función sapply aplica la función function(x) sum(is.na(x)) a cada columna de cust_churn_df, que cuenta el número de valores que faltan en cada columna. 
- El resultado de esta operación es un vector lógico que indica qué columnas tienen al menos un valor omitido. 
- La función *names* se utiliza para extraer los nombres de las columnas de este vector lógico. 
- Por último, se asigna a na_cols el vector de nombres de columnas con al menos un NA value.

In [ ]:
na_cols <- names(cust_churn_df)[sapply(cust_churn_df, function(x) sum(is.na(x))) > 0]
message("Number of NA's per variable:")
for (col in na_cols) {
  message(paste0(col, ": ", sum(is.na(cust_churn_df[[col]]))))
}

### Variable Objetivo: is_churned

- Variable objetivo: is_churned

In [ ]:
# distribución de frecuencias de la variable categórica objetivo is_churned
is_churned_counts <- table(cust_churn_df$is_churned)
is_churned_counts

In [ ]:
# proporción de cada categoría
prop.table(table(cust_churn_df$is_churned))

In [ ]:
# Create a data frame from the table
is_churned_df <- data.frame(is_churned = names(is_churned_counts), count = as.numeric(is_churned_counts))

# Create the plot using ggplot2
ggplot(is_churned_df, aes(x = is_churned, y = count)) +
  geom_bar(stat = "identity", fill = "blue") +
  labs(x = "Is Churned", y = "Count", title = "Churned vs Non-Churned Customers")

- La variable objetivo is_churned presenta un desbalanceo de clases:
    -  0: 74274     
    -  1: 29869

### Distribución de las variables categóricas

#### Device

In [ ]:
# Device
options(repr.plot.width = 20, repr.plot.height = 6)

pc1 <- ggplot(cust_churn_df, aes(x = device)) + 
  geom_bar() + 
  labs(title = "Device Distribution", x = "Device", y = "Count") +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))
pc1

- Observamos que hay un device 'unknown'.

In [ ]:
# Get the top 10 devices in descending order of count
top_devices <- cust_churn_df %>%
  count(device) %>%
  arrange(desc(n)) %>%
  head(10) %>%
  pull(device)

# Filter the data to include only the top 6 devices
cust_churn_top6 <- cust_churn_df %>%
  filter(device %in% top_devices)

# Create a bar plot of the top 6 devices in descending order of count
ggplot(cust_churn_top6, aes(x = fct_infreq(device), fill = device)) +
  geom_bar() +
  labs(title = "Top 10 Device Distribution", x = "Device", y = "Count", fill = "Device")


- Observamos que la marca de dispositivos más utilizada por los usuarios es Xiaomi, seguida de Samsung y Apple. 

#### City

In [ ]:
# City
options(repr.plot.width = 100, repr.plot.height = 6)

pc2 <- ggplot(cust_churn_df, aes(x = city)) + 
  geom_bar() + 
  labs(title = "City Distribution", x = "City", y = "Count") +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))

pc2

In [ ]:
# Get the top 10 cities in descending order of count
top_cities <- cust_churn_df %>%
  count(city) %>%
  arrange(desc(n)) %>%
  head(10) %>%
  pull(city)

# Filter the data to include only the top 10 cities
cust_churn_top5 <- cust_churn_df %>%
  filter(city %in% top_cities)

# Create a bar plot of the top 10 cities in descending order of count
ggplot(cust_churn_top5, aes(x = fct_infreq(city), fill = city)) +
  geom_bar() +
  labs(title = "Top 10 City Distribution", x = "City", y = "Count", fill = "City")

- Observamos que la ciudad con más usuarios es NCR, seguida de Bangalore y Mumbai. 

#### is_referral, given_permission_1, given_permission_2

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 3)

# is_referral
pc3 <- ggplot(cust_churn_df, aes(x = is_referral)) + 
      geom_bar() + 
      labs(title = "Referral Distribution", x = "Referral", y = "Count") 
# given_permission_1
pc4 <- ggplot(cust_churn_df, aes(x = given_permission_1)) + 
      geom_bar() + 
      labs(title = "Permission 1 Distribution", x = "Permission 1", y = "Count") 
# given_permission_2
pc5 <- ggplot(cust_churn_df, aes(x = given_permission_2)) + 
      geom_bar() + 
      labs(title = "Permission 2 Distribution", x = "Permission 2", y = "Count")
# arrange the plots in a 2x2 grid
grid.arrange(pc3, pc4, pc5, ncol = 3)

# distribución de frecuencias de la variable categórica objetivo is_churned
message('is_referral:')
table(cust_churn_df$is_referral)
message('given_permission_1:')
table(cust_churn_df$given_permission_1)
message('given_permission_2:')
table(cust_churn_df$given_permission_2)

- Observamos que en given_permission_1 y given_permission_2 hay un claro desbalanceo de clases, mientras que el desbalanceo de clases en is_referral no es tan acusado. 

### Distribución de las variables numéricas

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)

# first_payment_amount
p1 <- ggplot(cust_churn_df, aes(x = first_payment_amount)) +
      geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
      labs(title = "User’s first payment amount",
      subtitle = "núm.bins = 15", x = "first_payment_amount")
b1 <- ggplot(data = cust_churn_df, aes(x = "", y = first_payment_amount)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p1,b1)
# age
p2 <- ggplot(cust_churn_df, aes(x = age)) +
      geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
      labs(title = "User's age",
      subtitle = "núm.bins = 15", x = "age")
b2 <- ggplot(data = cust_churn_df, aes(x = "", y = age)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p2,b2)

# number_of_cards
p3 <- ggplot(cust_churn_df, aes(x = number_of_cards)) +
      geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
      labs(title = "Number of cards user owns",
      subtitle = "núm.bins = 15", x = "number_of_cards")
b3 <- ggplot(data = cust_churn_df, aes(x = "", y = number_of_cards)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p3,b3)

# payments_initiated
p4 <- ggplot(cust_churn_df, aes(x = payments_initiated)) +
      geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
      labs(title = "Number of payments initiated by user",
      subtitle = "núm.bins = 15", x = "payments_initiated")
b4 <- ggplot(data = cust_churn_df, aes(x = "", y = payments_initiated)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p4,b4)

# payments_failed
p5 <- ggplot(cust_churn_df, aes(x = payments_failed)) +
      geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
      labs(title = "Number of payments failed",
      subtitle = "núm.bins = 15", x = "payments_failed")
b5 <- ggplot(data = cust_churn_df, aes(x = "", y = payments_failed)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p5,b5)

# payments_completed
p6 <- ggplot(cust_churn_df, aes(x = payments_completed)) +
       geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
       labs(title = "Number of payments completed",
       subtitle = "núm.bins = 15", x = "payments_completed")
b6 <- ggplot(data = cust_churn_df, aes(x = "", y = payments_completed)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p6,b6)

# payments_completed_amount_first_7days
p7 <- ggplot(cust_churn_df, aes(x = payments_completed_amount_first_7days)) +
      geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
      labs(title = "Amount of payment completed in first 7 days of joining",
      subtitle = "núm.bins = 15", x = "payments_completed_amount_first_7days")
b7 <- ggplot(data = cust_churn_df, aes(x = "", y = payments_completed_amount_first_7days)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p7,b7)

# reward_purchase_count_first_7days
p8 <- ggplot(cust_churn_df, aes(x = reward_purchase_count_first_7days)) +
      geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
      labs(title = "Number of rewards claimed in first 7 days",
      subtitle = "núm.bins = 15", x = "reward_purchase_count_first_7days")
b8 <- ggplot(data = cust_churn_df, aes(x = "", y = reward_purchase_count_first_7days)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p8,b8)

# coins_redeemed_first_7days
p9 <- ggplot(cust_churn_df, aes(x = coins_redeemed_first_7days)) +
    geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
    labs(title = "Coins redeemed in first 7 days",
    subtitle = "núm.bins = 15", x = "coins_redeemed_first_7days")
b9 <- ggplot(data = cust_churn_df, aes(x = "", y = coins_redeemed_first_7days)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p9,b9)

# visits_feature_1
p10 <- ggplot(cust_churn_df, aes(x = visits_feature_1)) +
    geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
    labs(title = "Number of visits made by user to product feature 1",
    subtitle = "núm.bins = 15", x = "visits_feature_1")
b10 <- ggplot(data = cust_churn_df, aes(x = "", y = visits_feature_1)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p10,b10)

# visits_feature_2
p11 <- ggplot(cust_churn_df, aes(x = visits_feature_2)) +
    geom_histogram(bins = 15, fill = "#d94c9f", color = "white") +
    labs(title = "Number of visits made by user to product feature 2",
    subtitle = "núm.bins = 15", x = "visits_feature_2")
b11 <- ggplot(data = cust_churn_df, aes(x = "", y = visits_feature_2)) +
       stat_boxplot(geom = "errorbar" ,color = "#3c0023") +
       geom_boxplot(fill = "#d94c9f",color = "#3c0023") +
       coord_flip()
grid.arrange(p11,b11)

- Observamos un elevado número de outliers en todas las variables.
- En la mayoría de ellas, sin limpiar los datos, es muy difícil ver que distribución tienen.
- Podemos observar que la variable 'age' parece seguir una distribución normal. 

### Relación entre los atributos y la variable objetivo is_churned

#### Relación entre las variables numéricas i is_churned 

In [ ]:
# lista con las variables numéricas
num_vars <- c("first_payment_amount", 
              "age", 
              "number_of_cards", 
              "payments_initiated", 
              "payments_failed", 
              "payments_completed", 
              "payments_completed_amount_first_7days", 
              "reward_purchase_count_first_7days",
              "coins_redeemed_first_7days", 
              "visits_feature_1", 
              "visits_feature_2")

# df con las variables numéricas y la variable target
num_target_df <- cust_churn_df[, c(num_vars, "is_churned")]

##### Violinplots

In [ ]:
# Plot violin plots for each numerical variable and the target variable
options(repr.plot.width = 7, repr.plot.height = 6)

for (var in num_vars) {
  print(ggplot(num_target_df, aes_string(x = "is_churned", y = var)) + 
    geom_violin() + 
    ggtitle(paste("Violin plot of", var, "by is_churned")) +
    xlab("is_churned") + ylab(var) +
    geom_violin(aes(fill = is_churned)) + 
    theme(legend.position="none") +
    geom_boxplot(width=.1) + stat_summary(fun=mean, geom="point", shape=20, size=4, color="red", fill="red"))
}


- Podemos observar la distribución de cada variable numérica para los diferentes niveles (0 y 1) de la variable categórica objetivo (is_churned). 
- Podemos visualizar los outliers y comparar la media y la mediana de cada variable numérica respecto a cada nivel de la variable target.

##### Frequency polygon

Un frequency polygon es un gráfico que se utiliza para mostrar la forma de la distribución de un conjunto de datos continuos. Se trata de un gráfico lineal de una distribución de frecuencias en el que las frecuencias se trazan frente a los puntos medios de los intervalos de clase. La forma del polígono muestra el patrón general de la distribución.

En este caso, los polígonos de frecuencias se utilizan para visualizar la distribución de cada variable numérica en relación con la variable objetivo (is_churned). Los polígonos están coloreados por los niveles de la variable objetivo para visualizar cualquier diferencia en las distribuciones entre los dos niveles.

In [ ]:
options(repr.plot.width = 12, repr.plot.height = 8)

# Plot frequency polygons for each numerical variable and the target variable
for (var in num_vars) {
    print(
        ggplot(num_target_df, aes_string(x = var, color = 'is_churned')) + 
        geom_freqpoly(alpha = 0.8, size = 0.5, binwidth = 5) +
        ggtitle(paste("Distribution of", var, "and is_churned")) +
        xlab(var)  +
        scale_fill_manual(values = c("blue", "red")) 
        # + xlim(0, max(num_target_df[[var]]))
    )
}

##### Density plot

- Creamos gráficos de densidad para cada variable numérica y la variable objetivo (is_churned). 
- Esto es útil para visualizar la distribución de cada variable numérica en relación con la variable objetivo y puede proporcionar información sobre los posibles predictores del churn.

In [ ]:
# Define color scheme
color_scheme <- brewer.pal(9, "Set1")

# Plot density plots for each numerical variable and the target variable
for (var in num_vars) {
  print(
    ggplot(num_target_df, aes_string(x = var, fill = 'is_churned')) + 
    geom_density(alpha = 0.6) +
    ggtitle(paste("Density plot of", var, "and is_churned")) +
    xlab(var) + ylab("Density") +
    scale_fill_manual(values = c(color_scheme[1], color_scheme[2])) +
    theme_bw()
  )
}

#### Relación entre las variables categóricas y is_churned 

##### is_referral, given_permission_1 y given_permission_2 vs is_churned

Creamos gráficos de barras para cada variable categórica binaria del conjunto de datos y su relación con la variable objetivo "is_churned".

In [ ]:
# lista con las variables categóricas binarias
cat_binary_vars <- c('is_referral', 'given_permission_1', 'given_permission_2')

# df con las variables categóricas binarias y la variable target
cat_binary_target_df <- cust_churn_df[, c(cat_binary_vars, "is_churned")]

In [ ]:
# Plot bar charts for each categorical variable and the target variable
options(repr.plot.width = 5, repr.plot.height = 3)
colors <- brewer.pal(9, "Set1")
for (var in cat_binary_vars) {
  print(ggplot(cat_binary_target_df, aes_string(x = var, fill = "is_churned")) + 
    geom_bar(position = "dodge") +
    ggtitle(paste("Bar chart of", var, "and is_churned")) +
    xlab(var) + ylab("Count") +
    scale_fill_manual(values = colors))
}

- Los gráficos de barras resultantes muestran el recuento de cada variable categórica binaria y su relación con la variable objetivo.
- Observamos que para las 3 variables categóricas binarias, parece que el hecho que haya churn o no sí afecta. 

##### Device vs is_churned

In [ ]:
# Get the top 10 devices in descending order of count
top_devices <- cust_churn_df %>%
  count(device) %>%
  arrange(desc(n)) %>%
  head(10) %>%
  pull(device)

# Filter the data to include only the top 6 devices
cust_churn_top10 <- cust_churn_df %>%
  filter(device %in% top_devices)

# subset device and is_churned
cust_churn_top10_df <- cust_churn_top10 %>%
  dplyr::select(device, is_churned)

# Create a bar plot of the top 6 devices in descending order of count
options(repr.plot.width = 10, repr.plot.height = 3)

ggplot(cust_churn_top10_df, aes(x = fct_infreq(device), fill = is_churned)) +
  geom_bar(position="dodge") +
  labs(title = "Top 10 Device Distribution", x = "Device", y = "Count", fill = "is_churned")

- Parece ser que, en términos relativos, 'vivo' encabeza el número de abandonos. 

##### City vs is_churned

In [ ]:
# Get the top 10 cities in descending order of count
top_cities <- cust_churn_df %>%
  count(city) %>%
  arrange(desc(n)) %>%
  head(10) %>%
  pull(city)

# Filter the data to include only the top 6 devices
cust_churn_top5 <- cust_churn_df %>%
  filter(city %in% top_cities)

# subset device and is_churned
city_churn_top10_df <- cust_churn_top5 %>% dplyr::select(city, is_churned)

# Create a bar plot of the top 5 cities in descending order of count
ggplot(city_churn_top10_df, aes(x = fct_infreq(city), fill = is_churned)) +
  geom_bar(position="dodge") +
  labs(title = "Top 10 City Distribution", x = "City", y = "Count", fill = "Is churned?")

- Observamos que en 'Surat' hay más usuarios que hacen churn que los que permanecen. 

## Tratamiento de valores nulos

Mostramos las variables que contienen valores nulos y la cantidad de valores nulos:

In [ ]:
# select only columns with NA values
na_cols <- cust_churn_df[, colSums(is.na(cust_churn_df)) > 0]

# count NA values in each selected column
na_counts <- colSums(is.na(na_cols))

# print the number of NA values and variable names with NA values
for (i in seq_along(na_counts)) {
  cat(sprintf("Variable %s has %d NA values\n", names(na_counts)[i], na_counts[i]))
}

In [ ]:
cust_churn_df %>% 
  dplyr::select(num_vars) %>% 
  miss_var_summary()

- El 22.34% de los valores de reward_purchase_count_first_7days son NA values, mientras que first_payment_amount no presenta NA values. 

In [ ]:
num_vars_na<- c(
              "age", 
              "number_of_cards", 
              "payments_initiated", 
              "payments_failed", 
              "payments_completed", 
              "payments_completed_amount_first_7days", 
              "reward_purchase_count_first_7days",
              "coins_redeemed_first_7days", 
              "visits_feature_1", 
              "visits_feature_2")
num_df <- cust_churn_df[, c(num_vars_na)]

In [ ]:
# Another approach to visualising the missings in a dataset is to use the gg_miss_var plot:
options(repr.plot.width = 10, repr.plot.height = 3)

gg_miss_var(num_df)

- Observamos que reward_purchase_count_first_7days es la variable con más NA values (23264), seguida de visits_feature_1 y visits_feature_2, con 2646 NA values cada una. 

In [ ]:
# Echamos un vistazo a la distribución de los NA values
options(repr.plot.width = 20, repr.plot.height = 20)
md.pattern(num_df)

- Podemos observar la distribución de los NA values.
- Los NA values de la variable 'edad' se sitúan en las últimas filas de la columna. 

In [ ]:
md.pairs(num_df)

Por tanto, deberemos tratar los valores nulos de estas variables.

### Deletion and multiple imputation 

Para tratar los NA values, utilizaremos 2 técnicas:
- Eliminación
- Imputación múltiple

#### Eliminación

Como en la variable 'age' el número de filas con valores nulos es relativamente pequeño en comparación con el tamaño del dataset, simplemente eliminamos esas filas. 
- Tenemos 142 filas con valores nulos de un total de 104143 filas, por lo que éste podría ser un enfoque razonable (estaríamos eliminando un 0.13% del total de filas del dataset).

In [ ]:
# filas 
length(cust_churn_df$age)

In [ ]:
# NA values 
sum(is.na(cust_churn_df$age))

In [ ]:
# Create a copy of the cust_churn_df data frame
cust_churn_clean_df <- data.frame(cust_churn_df) 

In [ ]:
tracemem(cust_churn_df) == tracemem(cust_churn_clean_df)

- Esto creará un dataset llamado cust_churn_clean_df que es idéntico a cust_churn_df, pero cualquier cambio realizado en un dataset no afectará al otro.

- El nuevo dataset tiene la misma estructura y contenido que cust_churn_df, pero es un objeto completamente independiente en memoria. 



In [ ]:
sum(is.na(cust_churn_clean_df))
sum(is.na(cust_churn_clean_df$age))

In [ ]:
# remove rows with null values for the "age" variable
cust_churn_clean_df <- cust_churn_clean_df[!is.na(cust_churn_clean_df$age), ]
# check the number of rows with null values for the "age" variable
sum(is.na(cust_churn_clean_df$age))

- Ya no tenemos valores NA en la variable 'age'.

In [ ]:
# before removing NA's
summary(cust_churn_df$age)

In [ ]:
# after removing NA's
summary(cust_churn_clean_df$age)

In [ ]:
dim(cust_churn_clean_df)

- Hemos eliminado 142 filas del dataset.

#### Multiple imputation

- La imputación múltiple es una técnica estadística utilizada para tratar los NA values en un dataset imputando o sustituyendo los NA values por valores estimados basados en la información disponible en el dataset. 

- Esta técnica implica la creación de múltiples copias del dataset, donde los NA values se imputan con valores plausibles varias veces, generando una distribución de valores plausibles para cada NA value. 

- La imputación múltiple es una técnica útil, ya que puede ayudar a reducir el sesgo y aumentar la accuracy de los análisis estadísticos cuando faltan datos [4].

Además, la imputación múltiple ofrece varias ventajas con respecto a la simple eliminación o imputación de la media o mediana de los NA values [5]:

- Conserva más información: al crear imputaciones múltiples basadas en los datos observados, la imputación múltiple conserva más de la variabilidad original y de las relaciones entre las variables del dataset que otros métodos.

- Proporciona estimaciones más precisas: la imputación múltiple puede producir estimaciones de parámetros más precisas que los métodos de imputación única, que pueden subestimar la varianza y dar lugar a estimaciones sesgadas.

- Produce inferencias estadísticas válidas: la imputación múltiple puede mejorar la validez de las inferencias estadísticas al tener en cuenta la incertidumbre introducida por los datos que faltan. Esto puede ayudar a evitar el sesgo y las conclusiones incorrectas a causa de ignorar los datos que faltan.

- Reduce el sesgo y aumenta la potencia: la imputación múltiple puede reducir el sesgo y aumentar la potencia estadística al utilizar la información disponible para hacer predicciones más precisas de los datos que faltan.

En general, la imputación múltiple es un método más flexible y sólido para tratar los datos que faltan que ofrece varias ventajas sobre métodos más sencillos como la eliminación o la imputación de la media o la mediana.

Por tanto, aplicaremos la imputación múltiple a las siguientes variables:
- number_of_cards, payments_initiated, payments_failed, payments_completed, payments_completed_amount_first_7days y coins_redeemed_first_7days: tienen 472 valores NA (cantidad moderada de NA values).

- reward_purchase_count_first_7days: tiene 23264 NA values (cantidad de NA values relativamente grande).

- visits_feature_1 y visits_feature_2: tienen 2646 valores NA (cantidad moderada de NA values).

In [ ]:
# create a vector of variable names to be imputed
vars_to_impute <- c(
              "number_of_cards", 
              "payments_initiated", 
              "payments_failed", 
              "payments_completed", 
              "payments_completed_amount_first_7days", 
              "reward_purchase_count_first_7days",
              "coins_redeemed_first_7days", 
              "visits_feature_1", 
              "visits_feature_2")

# create the imputation model using mice()
impute_model <- mice(cust_churn_clean_df[,vars_to_impute], m = 5, method = "pmm", seed = 123)

# impute the missing values using complete()
imputed_data <- complete(impute_model)

In [ ]:
head(imputed_data, 3)

In [ ]:
summary(imputed_data)
message()
dim(imputed_data)
message()
sum(is.na(imputed_data))

In [ ]:
# añadimos las variables a las que les hemos aplicado la multiple imputation al df limpio de NA values
for (var in vars_to_impute) {
    cust_churn_clean_df[[var]] <- imputed_data[[var]] 
}

In [ ]:
sum(is.na(cust_churn_clean_df))

- Ya no tenemos ningún NA value en nuestro dataset.
- Tenemos un dataset totalmente limpio.

In [ ]:
dim(cust_churn_clean_df)

- Recordemos que solo hemos borrado 142 filas de un total de 104143.

In [ ]:
head(cust_churn_clean_df)

## Correlation Analysis

- Primero realizaremos un análisis de correlaciones. 
- El análisis de correlación nos puede ayudar a identificar las variables que están significativamente asociadas con la variable objetivo is_churned.
- Esto nos ayudará seleccionar las variables más relevantes para incluirlas en el modelo de regresión logística.

### Correlación entre variables numéricas y la variable objetivo categórica is_churned

La correlación biserial mide la fuerza de la asociación entre una variable continua y una binaria, en este caso entre cada variable numérica y la variable categórica objetivo is_churned.


In [ ]:
# Select only numerical variables
numerical_vars <- c("first_payment_amount", "age", "number_of_cards", "payments_initiated", "payments_failed",
                    "payments_completed", "payments_completed_amount_first_7days", "reward_purchase_count_first_7days",
                    "coins_redeemed_first_7days", "visits_feature_1", "visits_feature_2")

df_num <- cust_churn_clean_df[numerical_vars]

# create an empty vector to store the results
correlations <- c()

# loop through each numerical variable and calculate the biserial correlation
for (var in colnames(df_num)) {
  # calculate biserial correlation between the current variable and the target variable
  cor <- biserial.cor(df_num[[var]], cust_churn_clean_df$is_churned)
  correlations <- c(correlations, cor)
}

# print the results
cor_var <- data.frame(variable = colnames(df_num), correlation = correlations)
cor_var

- Observando los resultados, podemos ver que varias de las variables numéricas tienen una correlación positiva con la variable objetivo is_churned, lo que indica que a medida que estas variables aumentan, la probabilidad de que un cliente abandone también aumenta. Estas variables son: first_payment_amount, number_of_cards, payments_initiated, payments_completed_amount_first_7days, reward_purchase_count_first_7days y coins_redeemed_first_7days.

- Por otro lado, podemos ver que la edad tiene una correlación negativa con is_churned, lo que sugiere que, a medida que los clientes envejecen, es menos probable que hagan churn. 

- payments_failed y visits_feature_2 tienen la menor correlación con is_churned, lo que indica que están débilmente asociadas con la variable objetivo.

En conclusión, el análisis de correlación biserial proporciona información útil sobre la relación entre las variables numéricas y la variable categórica objetivo is_churned, y puede ayudar a identificar qué variables pueden ser las más importantes para predecir el abandono de los clientes. Sin embargo, es importante señalar que la correlación no implica necesariamente causalidad, por lo que se necesitan más análisis para confirmar estas relaciones y establecer la causalidad.

### Asociación entre variables categóricas y la variable objetivo categórica is_churned

- Dado que tenemos variables categóricas, no podemos calcular una correlación entre ellas y la variable objetivo is_churned. En su lugar, podemos utilizar la prueba chi-squared para determinar si existe una asociación estadísticamente significativa entre cada variable categórica y is_churned. 

- Este test mide el grado de dependencia entre las dos variables categóricas. El test calcula un estadístico chi-squared y un p-value que indica la probabilidad de observar un resultado tan extremo o más extremo que el observado, suponiendo que la hipótesis nula es cierta (es decir, no hay asociación entre las dos variables).

In [ ]:
# Select only categorical variables
cat_vars <- c('device', 'city', 'is_referral', 'given_permission_1', 'given_permission_2')
# binary_cat_vars <- c('is_referral', 'given_permission_1', 'given_permission_2')

df_cat <- cust_churn_clean_df[cat_vars]

In [ ]:
# Calculate chi-square test of independence for each variable
cat_var_cor <- sapply(cust_churn_clean_df[cat_vars], function(x) {
  chisq.test(table(x, cust_churn_clean_df$is_churned))$statistic
})

# Calculate p-value for each variable
cat_var_pval <- sapply(cat_var_cor, function(x) {
  pvalue <- pchisq(x, df = 1, lower.tail = FALSE)
  return(pvalue)
})

# Combine test statistics and p-values into a data frame
cat_var_cor_df <- data.frame(variable = names(cat_var_cor), chi_square_statistic = cat_var_cor, p_value = cat_var_pval)

cat_var_cor_df


- Podemos ver que el estadístico chi-cuadrado más alto es para "city" (seguido de "device"), lo que indica una asociación más fuerte con la variable objetivo "is_churned" en comparación con las otras variables categóricas. Sin embargo, también debemos comprobar el p-value de cada prueba para determinar si la asociación es estadísticamente significativa al nivel de significación deseado.

- Podemos ver que todos los p-values son inferiores a 0.05, lo que indica una relación estadísticamente significativa entre cada una de las variables categóricas y la variable objetivo 'is_churned'.

- Nota: el warning indica que la aproximación chi-cuadrado utilizada en la prueba puede no ser exacta debido al tamaño de los datos o a la distribución de las variables. Se trata de un problema habitual cuando se utilizan pruebas de chi-cuadrado con conjuntos de datos de gran tamaño.


## Mutual information

- Esta prueba mide la dependencia mutua entre dos variables, ya sean continuas o categóricas. 
- A diferencia de la correlación, la información mutua puede captar relaciones no lineales entre variables. 
- Es una prueba útil para identificar predictores potenciales que tengan una fuerte asociación con la variable objetivo. 
- Además, puede ayudar a identificar términos de interacción relevantes (combinaciones de dos o más predictores) que pueden tener un fuerte impacto en la variable objetivo. 
- Por lo tanto, el uso de la información mutua puede ayudarnos a construir un modelo de regresión logística más preciso y sólido.

- Las variables más importantes serán aquellas que aporten más entropía a nuestro dataset. 
- Como más entropía, más importancia. 

**Information Gain Algorithms**:

Con tal de ver la información mutua entre el target y las covariables se aplican los tres algoritmos detallados a continuación:

- "infogain" : ${H(Class)} + H(Attribute) − H(Class, Attribute)$
    - Este es el tipo de ganancia de información más utilizado. 
    - Se basa en la diferencia entre la entropía de la variable objetivo y la entropía condicional de la variable objetivo dada la variable predictora.

- "gainratio" : $H(Class) + H(Attribute) − H(Class, Attribute) / H(Attribute) $
    - Este tipo de ganancia de información se utiliza para ajustar el sesgo que puede producirse cuando una variable predictora tiene muchos valores o niveles distintos. 
    - Divide la ganancia de información por la información dividida, que es una medida de la diversidad de los valores de la variable predictora.

- "symuncert" : $2 * (H(Class) + H(Attribute) − H(Class, Attribute)) / (H(Attribute) + H(Class)) $
    - Este tipo de ganancia de información mide la información mutua entre la variable objetivo y la variable predictora, y la ajusta por la entropía de ambas variables.
    
La elección del tipo de ganancia de información depende del problema específico y de la naturaleza de las variables predictoras. En general, infogain es un buen punto de partida y funciona bien con variables categóricas, mientras que gain_ratio y symmetrical_uncertainty son más adecuadas para variables categóricas continuas o de alta cardinalidad.

In [ ]:
# información mútua 
i <- information_gain(formula = is_churned ~ .,
                      data = cust_churn_clean_df[, !(names(cust_churn_clean_df) %in% c("is_churned_numeric", "user_id"))],
                      type = 'infogain')
i  

# variables que tienen más importancia en nuestro dataset   
ggplot(i, aes(x=attributes, y=importance)) + 
  geom_bar(stat = "identity") +
  coord_flip()


- La columna "atributos" enumera todas las variables (predictores) del dataset, y la columna "importancia" muestra la puntuación de importancia de cada variable, con respecto a la variable objetivo "is_churned". 

- La puntuación de importancia se mide utilizando la métrica de ganancia de información, que indica la reducción de la incertidumbre de la variable objetivo, dado el conocimiento de una determinada variable predictora.

- Basándonos en las puntuaciones de la importancia, podemos ver que entre los predictores, "payments_completed_amount_first_7days" tiene la puntuación de importancia más alta (0.073), seguida de "first_payment_amount" (0.058). Es probable que estas variables sean predictores importantes de la pérdida de clientes y podría ser interesante tenerlas en cuenta en el modelo de regresión logística.

In [ ]:
### gainratio: variable target con dataset
ig = information_gain(formula = is_churned ~ .,
                      data = cust_churn_clean_df[, !(names(cust_churn_clean_df) %in% c("is_churned_numeric", "user_id"))],
                      type = 'gainratio')
ig

ggplot(ig, aes(x=attributes, y=importance)) + 
  geom_bar(stat = "identity") +
  coord_flip()

In [ ]:
### gainratio: variable target con dataset
is = information_gain(formula = is_churned ~ .,
                      data = cust_churn_clean_df[, !(names(cust_churn_clean_df) %in% c("is_churned_numeric", "user_id"))],
                      type = 'symuncert')

is

ggplot(is, aes(x=attributes, y=importance)) + 
  geom_bar(stat = "identity") +
  coord_flip()

- Ahora tenemos una visión de cuáles son las variables más importantes de nuestro dataset. 

- Entre ellas destacan:
    - payments_completed_amount_first_7days	
    - first_payment_amount	
    - coins_redeemed_first_7days
    - reward_purchase_count_first_7days
    - payments_completed
    
Por tanto, es probable que estas variables sean predictores importantes de la pérdida de clientes y podría ser interesante tenerlas en cuenta en el modelo de regresión logística.

## PCA

- Hacemos el PCA para limpiar las variables y ver con qué variables nos quedamos (solo lo haremos con las variables numéricas).

- Obtenemos tantas componentes como variables numéricas tengamos.

- Sirve para empezar a limpiar variables.

- Posteriormente, con las variables que nos quedemos, volvemos a hacer correlaciones para ver cuáles son las que más correlacionan con la variable objetivo, y también para ver como se comportan entre ellas, ya que no queremos poner información redundante dentro del modelo. Eso también ayuda a limpiar.

- acp = princomp(data_macro_cor,cor=TRUE)

    - Tomamos como referencia correlaciones porque es muy importante que cuando tengamos diferentes unidades una no pese más que la otra.
    - Si el objetivo es analizar las relaciones entre variables, princomp puede ser más apropiado.
    - Si el objetivo es analizar los patrones en las variables, prcomp puede ser más apropiado.

In [ ]:
data_cor = data.frame(cust_churn_clean_df$first_payment_amount,
                     cust_churn_clean_df$age,
                     cust_churn_clean_df$number_of_cards,
                     cust_churn_clean_df$payments_initiated,
                     cust_churn_clean_df$payments_failed,
                     cust_churn_clean_df$payments_completed,
                     cust_churn_clean_df$payments_completed_amount_first_7days,
                     cust_churn_clean_df$reward_purchase_count_first_7days,
                     cust_churn_clean_df$coins_redeemed_first_7days,
                     cust_churn_clean_df$visits_feature_1,
                     cust_churn_clean_df$visits_feature_2
                     )
# matriz de correlación 
round(cor(data_cor),2)

# PCA
acp = princomp(data_cor,cor=TRUE)
# resumen del PCA
summary(acp)

# sd
print('desviación estandard componente1:')
sd(predict(acp)[,1:1])

# varianza
print('varianza componente1:')
var(predict(acp)[,1:1])

- Obtenemos un total de 11 componentes (tantas como variables tengo sin contar las categóricas).

- La 1ª componente es la que más varianza nos va a explicar: ya lo resume, ya hace la combinación lineal entre nuestras variables que explique todas esas variables que le hemos metido. 

- Miramos la Proportion of Variance (una medida de la cantidad de variación en los datos originales que es capturada por cada uno de los componentes principales) de la componente 1 y 2: 0.29 + 0.16 = 0.45. Es decir, la comp1 y 2 nos explican el 45% de la variación de los datos. 

- Añadimos la componente 3: 0.298 + 0.167 + 0.113 = 0.579. La comp 1, 2 y 3 nos explican el 58% de la variación de los datos.

- Con un 58%, para empezar a generar un modelo y comenzar a limpiar variables ya nos sirve. 

In [ ]:
# A través de esta función tendriamos las puntuaciones en las componentes principales
# Nos es útil para comparar  como para usar estas componentes sustituyendolas por las variables originales
predict(acp)
# acp$scores #otra forma de llamarlos

- La función predict() en R se utiliza para hacer predicciones basadas en un modelo. 

- En el caso, la función predict() se aplica a un objeto PCA obtenido de la función princomp(), lo que significa que devolverá las puntuaciones de componentes principales predichas para las observaciones del conjunto de datos original.

- Las puntuaciones de los componentes principales son las coordenadas de las observaciones en el nuevo sistema de coordenadas definido por los componentes principales. 

- La puntuación de la primera componente principal es la proyección de cada observación sobre la primera componente principal, la puntuación de la segunda componente principal es la proyección de cada observación sobre la segunda componente principal, y así sucesivamente.

- Es decir, para cada una de mis observaciones, que valor tendría esa nueva variable (esa componente), para cada una las componentes. A partir de esto luego podremos dibujarlo en los ejes de coordenadas mediante el biplot(). Esto nos ayudará a saber qué variables son las que mayor peso tienen en la componente 1 o en la componente 2. 

In [ ]:
loadings(acp)

- Loadings son los pesos: nos dice qué variables son las que realmente están aportando mayor información en esta componente. Los pesos son los coeficientes que describen la combinación lineal de las variables originales que componen cada componente principal.

- Nos quedamos de la componente 1 con el reward_purchase_count_first_7days (0.486) que es la que mayor peso tiene, de la componente 2 con el first_payment_amount (0.573) y de la componente 3 visits_feature_1 (0.698). 

- Seleccionaremos también payments_initiated (0.469) y payments_completed_amount_first_7days (0.523) ya que pueden ser interesantes. 

- (Se miran tanto las positivas como las negativas, las que tengan más peso ya sea en + o en -).

- Ya hemos hecho una criba de variables.

-  Queremos saber qué variables contribuyen más a esas componentes principales. Con solo estas, ya vamos a tener una mínima explicabilidad. ¿Esto nos será suficiente para tener un buen modelo? Vamos a verlo.

In [ ]:
options(repr.plot.width = 10, repr.plot.height = 6)
plot(acp)

Barplot
- La suma de las varianzas tiene que sumar 11: entre las 3 primeras componentes ya vemos que casi todo queda explicado, las últimas son residuales.

Biplot:
- En este caso no lo graficamos ya que el programa 'peta', se queda sin memoria. 
- Dibujamos las variables en los ejes. 
- Gráfico de dispersión de las puntuaciones de los componentes principales que incluye los pesos (o coeficientes) de las variables originales en los componentes principales. Las cargas se representan mediante flechas que indican la dirección y la magnitud de la contribución de cada variable a los componentes principales. La longitud de la flecha corresponde a la magnitud de la carga, y la dirección de la flecha indica la dirección de la carga.
- Qué nuevo valor tienen la comp1 y la comp2 para dibujarlo en el eje de coordenadas. 
- Podemos obervar si una variable está más relacionada con la comp1 o la comp2.
- Es una representación extra que podemos hacer para visualizarlo. 

### Matriz de componentes

In [ ]:
mcomponentes = cor(data_cor,predict(acp)[,1:3]) # Matriz de componentes
mcomponentes


apply(mcomponentes*mcomponentes,1,sum)

- Los pesos de los componentes muestran las relaciones lineales entre cada variable original y los 3 componentes. Los valores positivos indican una relación positiva entre la variable original y el componente, mientras que los valores negativos indican una relación negativa.

- Podemos observar que varias variables tienen pesos elevados en el primer componente (Comp.1), como reward_purchase_count_first_7days, payments_initiated y payments_completed. Esto sugiere que estas variables impulsan el primer componente, que podría captar algún aspecto importante de la variación de los datos.

- El segundo componente (Comp.2) tiene pesos positivos para la variable first_payment_amount y payments_completed_amount_first_7days. Esto sugiere que este componente podría estar captando alguna relación entre estas variables en los datos.

- El tercer componente (Comp.3) tiene pesos positivos elevados para visits_feature_1 y visits_feature_2. Esto sugiere que este componente podría estar capturando alguna relación entre estas variables en los datos.

- Por tanto, los resultados de este PCA sugieren que existen varias relaciones subyacentes en los datos que podrían ser captadas por estos 3 componentes.

In [ ]:
# payments_initiated
(0.85029154)^2 + (-0.43518697)^2 + (-0.039441047)^2

- Por ejemplo. la comp1, la comp2 y la comp3 me están explicando el 91% de la variación del payments_initiated. Me está conservando mucha información. Estas nuevas variables abstractas me están explicando mucho. 
- Por tanto, un 91% de la varianza de payments_initiated viene explicada por la componente 1, 2 y 3.

- Para las otras variables de interés:

In [ ]:
# payments_completed
(0.82683019)^2 + (-0.41842746)^2 + (-0.081579348)^2

In [ ]:
# payments_completed_amount_first_7days	
(0.57048000)^2 + (0.71001877)^2 + (0.006048129)^2

In [ ]:
# reward_purchase_count_first_7days
0.88004924^2 + (-0.17401327)^2 + (-0.042138734)^2

In [ ]:
# first_payment_amount
(0.27949804)^2 + (0.77722073)^2 + (0.026602194)^2

In [ ]:
# visits_feature_1	
(0.10019928)^2 + (-0.02252901)^2 + (0.779061892)^2

In [ ]:
# visits_feature_2
(0.10625789)^2 + (-0.02214432)^2 + (0.736241989)^2

## ANOVA

- La ANOVA nos dice cómo cuantificar la pertenencia a una de las categorías.
- La ANOVA puede ser una herramienta útil para identificar si existen diferencias significativas en las características numéricas entre los grupos de clientes que abandonaron y los que no.

In [ ]:
var_names <- c("first_payment_amount", "age", "number_of_cards", "payments_initiated", 
               "payments_failed", "payments_completed", "payments_completed_amount_first_7days", 
               "reward_purchase_count_first_7days", "coins_redeemed_first_7days", 
               "visits_feature_1", "visits_feature_2", "is_churned")
data_anova <- cust_churn_clean_df[var_names]

### Primeras suposiciones

In [ ]:
message('first_payment_amount')
data_anova %>% # pasamos los datos en el objeto data_anova como entrada a la siguiente operación
    group_by(is_churned) %>%  # agrupamos los datos en el objeto boston_aov_df por la variable is_churned  
    # calculamos la media y la desviación estándar de la variable first_payment_amount para cada grupo de is_churned  
    summarize(media = mean(first_payment_amount), desv = sd(first_payment_amount))

message('age')
data_anova %>% 
    group_by(is_churned) %>%  
    summarize(media = mean(age), desv = sd(age))

message('number_of_cards')
data_anova %>% 
    group_by(is_churned) %>%  
    summarize(media = mean(number_of_cards), desv = sd(number_of_cards))

message('payments_initiated')
data_anova %>%
group_by(is_churned) %>%
summarize(media = mean(payments_initiated), desv = sd(payments_initiated))

message('payments_failed')
data_anova %>%
group_by(is_churned) %>%
summarize(media = mean(payments_failed), desv = sd(payments_failed))

message('payments_completed')
data_anova %>%
group_by(is_churned) %>%
summarize(media = mean(payments_completed), desv = sd(payments_completed))

message('payments_completed_amount_first_7days')
data_anova %>%
group_by(is_churned) %>%
summarize(media = mean(payments_completed_amount_first_7days), desv = sd(payments_completed_amount_first_7days))

message('reward_purchase_count_first_7days')
data_anova %>%
group_by(is_churned) %>%
summarize(media = mean(reward_purchase_count_first_7days), desv = sd(reward_purchase_count_first_7days))

message('coins_redeemed_first_7days')
data_anova %>%
group_by(is_churned) %>%
summarize(media = mean(coins_redeemed_first_7days), desv = sd(coins_redeemed_first_7days))

message('visits_feature_1')
data_anova %>%
group_by(is_churned) %>%
summarize(media = mean(visits_feature_1), desv = sd(visits_feature_1))

message('visits_feature_2')
data_anova %>%
group_by(is_churned) %>%
summarize(media = mean(visits_feature_2), desv = sd(visits_feature_2))

- Parece que los todos grupos tienen valores diferentes de media y desviación típica para cada la variable.

- Basándonos únicamente en este análisis, no podemos afirmar que, que el cliente haga churn se vea afectado por estas variables. 

- Observando el gráficos de violín (vistos en apartados anteriores) así como las medias y las desviaciones estándar por categoría parece ser que hay una notable diferencia entre las distintas variables numéricas y si el cliente hace churn o no. 

- Las distribución de los datos no parece una distribución normal (lo hemos visto en apartados anteriores mediante violinplots), aunque esto lo corroboraremos en el siguiente punto mediante tests de normalidad como el Anderson-Darling test. 

### Comprobación de condiciones 

Comprobamos las condiciones para poder aplicar el test ANOVA.

ANOVA (Analysis of Variance) es una técnica estadística que se utiliza para dos o más grupos para comprobar si existe una diferencia entre sus valores medios [1],[2].

Las suposiciones de ANOVA son:
- La distribución de los datos es normal
- La varianza es constante entre los grupos
- Las observaciones son independientes entre sí


#### Normalidad

- Visualización: Q-Q Plot
- Prueba estadística: se utiliza el test de normalidad de Anderson-Darling ya tenemos más de 50 muestras:
    - H0: La variable presenta una distribución normal
    - H1: La variable presenta una distribución no normal
    - p-value > 0.05: No rechazar H0 (normal)
    - p-value < 0.05: Rechazar H0


In [ ]:
# Create normal probability plots 
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))

message('first_payment_amount')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'first_payment_amount'])
  qqline(subset(data_anova, is_churned == i)[,'first_payment_amount'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))
message('age')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'age'])
  qqline(subset(data_anova, is_churned == i)[,'age'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))
message('number_of_cards')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'number_of_cards'])
  qqline(subset(data_anova, is_churned == i)[,'number_of_cards'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))
message('payments_initiated')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'payments_initiated'])
  qqline(subset(data_anova, is_churned == i)[,'payments_initiated'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))
message('payments_failed')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'payments_failed'])
  qqline(subset(data_anova, is_churned == i)[,'payments_failed'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))

message('payments_completed')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'payments_completed'])
  qqline(subset(data_anova, is_churned == i)[,'payments_completed'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))
message('payments_completed_amount_first_7days')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'payments_completed_amount_first_7days'])
  qqline(subset(data_anova, is_churned == i)[,'payments_completed_amount_first_7days'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))

message('reward_purchase_count_first_7days')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'reward_purchase_count_first_7days'])
  qqline(subset(data_anova, is_churned == i)[,'reward_purchase_count_first_7days'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}


In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))

message('coins_redeemed_first_7days')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'coins_redeemed_first_7days'])
  qqline(subset(data_anova, is_churned == i)[,'coins_redeemed_first_7days'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))

message('visits_feature_1')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'visits_feature_1'])
  qqline(subset(data_anova, is_churned == i)[,'visits_feature_1'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)
par(mfrow = c(1,2))

message('visits_feature_2')
for(i in unique(data_anova$is_churned)){
  qqnorm(subset(data_anova, is_churned == i)[,'visits_feature_2'])
  qqline(subset(data_anova, is_churned == i)[,'visits_feature_2'])
  title(paste("\t\t\t\t\t\tChurn: ",i))
}


In [ ]:
# Define the variable names
var_names_ad <- c("first_payment_amount", "age", "number_of_cards", "payments_initiated", 
               "payments_failed", "payments_completed", "payments_completed_amount_first_7days", 
               "reward_purchase_count_first_7days", "coins_redeemed_first_7days", 
               "visits_feature_1", "visits_feature_2")

# Subset the data
data_ad <- cust_churn_clean_df[var_names_ad]

# Loop through the variables and compute the Anderson-Darling test
for (var in var_names_ad) {
  for (churn in unique(data_anova$is_churned)) {
    ad_test <- ad.test(subset(data_anova, is_churned == churn)[[var]])
    cat(paste("Variable:", var, "Churn:", churn, "\n"))
    cat(paste("Statistic:", ad_test$statistic, "\n"))
    cat(paste("p-value:", ad_test$p.value, "\n"))
    cat("\n")
  }
}


- Observamos que el p-value < 0.05 para ambos grupos de todas las variables numéricas.  
- El test de normalidad de Anderson Darling rechaza la hipótesis de normalidad cuando el p-value ≤ 0.05.  
- Fallar la prueba de normalidad permite afirmar con un 95% de confianza que los datos no se ajustan a la distribución normal.

No obstante, aunque los datos NO pasen la prueba de normalidad, es importante comprobar otros supuestos del ANOVA, como la homocedasticidad.

#### Homocedasticidad

Homocedasticidad: las varianzas son iguales en todos los grupos.
-  Bartlett’s test: 
    - Para verificar el supuesto de que las varianzas son iguales en todos los grupos.
    - Compara las varianzas de k muestras, donde k puede ser más de dos muestras [3]. 
    - Los datos deben estar distribuidos normalmente.
    - Hipótesis nula H0: todas las varianzas de las poblaciones son iguales.
    - Hipótesis alternativa H1: al menos dos de ellas difieren.

In [ ]:
# Bartlett’s test with one independent variable (is_churned):
for (var in var_names_ad) {
  res <- bartlett.test(get(var) ~ is_churned, data = data_anova)
  print(paste("Variable:", var))
  print(res)
}

- Podemos observar que el p-value no es superior al nivel de significancia (p-value < 0.05). 

- Por tanto, podemos rechazar la hipótesis nula y podemos concluir que las varianzas no son iguales. 



#### Independencia en las muestras

- La independencia de las muestras se refiere al supuesto de que las observaciones de cada muestra no están influidas por las observaciones de otras muestras.

- Que las muestras sean independientes o no depende del contexto de los datos y de cómo se hayan recogido. 

- Por ejemplo, si los datos se recogieron en distintas ciudades y cada ciudad representa una muestra independiente, se puede suponer que las observaciones de cada ciudad son independientes de las observaciones de otras ciudades.

- Suponemos que las muestras son independientes.

- Después de realizar los supuestos, vemos que no se cumplen los supuestos de normalidad y homocedasticidad.
- Por tanto NO podemos aplicar el test ANOVA.
- Deberemos usar alternativas no-paramétricas, como por ejemplo, el test de Kruskal-Wallis, que no asume la normalidad ni la igualdad de varianzas entre grupos.


## Kruskall - Wallis test

- La prueba de Kruskal-Wallis es un método no paramétrico para testear la hipótesis nula de que las medianas de dos o más grupos son iguales. 

- Puede utilizarse para determinar si las diferentes variables númericas que tenemos son significativamente diferentes en los dos niveles de is_churned (0 y 1).

- Si la prueba de Kruskal-Wallis muestra una diferencia significativa en las diferentes variables númericas entre los dos niveles de is_churned, indicaría que el hecho de que el cliente haga churn o no, tiene un efecto significativo en en esas variables, y podríamos considerar incluirlas como predictoras en el modelo de regresión logística.

In [ ]:
# Perform the Kruskal-Wallis test
for (var in var_names_ad) {
  res <- kruskal.test(get(var) ~ is_churned, data = data_anova)
  print(paste("Variable:", var))
  print(res)
}

- Como el p-value < 0.05 para todas las variables numéricas, podemos rechazar la hipótesis nula y concluir que existen pruebas significativas de que las medianas de las variables numéricas son diferentes entre los dos niveles de is_churned.
- Esto indica que existe una diferencia en los valores medianos de las variables numéricas en función si el cliente hace churn o no. 
- Basándose en este resultado, puede ser apropiado considerar la inclusión estas variables como predictoras en nuestro modelo de regresión logística. No obstante, qué variables incluiremos en el modelo lo acabaremos de decidir posteriormente. 

## Construcción del modelo de regresión logística

### División del dataset en train y test

Cuando empezamos a generar el modelo lo primero que hacemos es dividir el dataset en test y train:


In [ ]:
data_lr = cust_churn_clean_df
set.seed(123)
inTrain = createDataPartition(y = cust_churn_clean_df$is_churned
                              ,p = 0.7 # 70% train, 30% test
                              , list = FALSE
                              , times = 1)

data_train  = data_lr[inTrain, ]
data_test = data_lr[-inTrain, ]

In [ ]:
dim(data_train)
dim(data_test)

- Estos conjuntos de datos se utilizarán para entrenar y testear el modelo de regresión logística, donde el modelo se ajustará a los datos de entrenamiento y se utilizará para hacer predicciones sobre los datos de prueba. Esto le permitirá evaluar el rendimiento del modelo en datos no observados y asegurarse de que el modelo generaliza bien con nuevos datos.

### Selección de variables predictoras

- Posteriormente al PCA, con las variables que nos hemos quedado, cuando ya hemos detectado los predictores que queremos entrar al modelo, debemos confirmar que estos son independientes entre ellos: buscamos si puede haber problemas multicolinealidad.

- Volvemos a generar las correlaciones solo con las variables numéricas que nos interesan. 

Variables numéricas seleccionadas entre el análisis de correlación, el análisis de información mútua y el PCA:
- first_payment_amount
- payments_initiated
- payments_completed
- reward_purchase_count_first_7days
- payments_completed_amount_first_7days
- visits_feature_1
- visits_feature_2

In [ ]:
pca_var <- c("first_payment_amount", 
             "payments_initiated", 
             "payments_completed", 
             "reward_purchase_count_first_7days", 
             "payments_completed_amount_first_7days",
             "visits_feature_1", 
             "visits_feature_2"            
            )
pca_data <- cust_churn_clean_df[pca_var]

In [ ]:
# create an empty vector to store the results
correlations_pca <- c()

# loop through each numerical variable and calculate the biserial correlation
for (var in colnames(pca_data)) {
  # calculate biserial correlation between the current variable and the target variable
  cor <- biserial.cor(pca_data[[var]], cust_churn_clean_df$is_churned)
  correlations_pca <- c(correlations_pca, cor)
}

# print the results
cor_var_pca <- data.frame(variable = colnames(pca_data), correlation = correlations_pca)
cor_var_pca

- Parece ser, por ejemplo, que elegir reward_purchase_count_first_7days, payments_completed_amount_first_7days, first_payment_amount y payments_initiated puede ser una buena opción (como ya habíamos visto anteriormente). 

- Como en el caso de la regresión lineal, debemos comprobar si hay multicolinealidad en el modelo. A continuación comprobaremos que efectivamente no presenten problemas de multicolinealidad.

## Modelos de Regresión Logística

### Modelo 1

In [ ]:
# Fit logistic regression model on the training data
logitMod_1 <- glm(is_churned ~ first_payment_amount + 
                   payments_initiated + 
                   payments_completed + 
                   reward_purchase_count_first_7days + 
                   payments_completed_amount_first_7days + 
                   visits_feature_1 + visits_feature_2, 
                 data=data_train, family=binomial(link="logit"))
# summary 
summary(logitMod_1)

# Make predictions on the test data
m1_prob <- predict(logitMod_1, newdata=data_test, type = 'response')
m1_pred <- ifelse(m1_prob > 0.4, 1, 0)

# Compute confusion matrix
cm <- confusionMatrix(factor(m1_pred), factor(data_test$is_churned))
print(cm)

In [ ]:
# Calculate the VIF for each predictor in the model
VIF(logitMod_1)

Creamos un gráfico para ver que valores VIF superan el 5:

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 4)

par(mar = c(3,17,4,1) + 0.1, mgp = c(3,1,0), cex.lab=0.9)

# create vector of VIF values
vif_values <- VIF(logitMod_1)

#create horizontal bar chart to display each VIF value
barplot(vif_values, main = "VIF Values", horiz = TRUE, col = "steelblue", las=2, xlim = c(0, 5))

# add vertical line at 5
abline(v = 5, lwd = 3, lty = 2)

- Como podemos observar, todas las variables del modelo tienen un VIF muy por debajo de 5.
- Parece ser que las variables candidatas a ser predictoras (first_payment_amount, payments_initiated, payments_completed, reward_purchase_count_first_7days, payments_completed_amount_first_7days, visits_feature_1, visits_feature_2) no presentan problemas de multicolinealidad.

### Modelo 2

In [ ]:
# Fit logistic regression model on the training data
logitMod_2 <- glm(is_churned ~ first_payment_amount + payments_completed, 
                  data=data_train, family=binomial(link="logit"))
# summary 
summary(logitMod_2)

# Make predictions on the test data
m2_prob <- predict(logitMod_2, newdata=data_test, type = 'response')
m2_pred <- ifelse(m2_prob > 0.3, 1, 0)

# Compute confusion matrix
cm_2 <- confusionMatrix(factor(m2_pred), factor(data_test$is_churned))
print(cm_2)

### Modelo 3

In [ ]:
# Fit logistic regression model on the training data
logitMod_3 <- glm(is_churned ~  first_payment_amount + payments_initiated + reward_purchase_count_first_7days, 
                  data=data_train, family=binomial(link="logit"))
# summary 
summary(logitMod_3)

# Make predictions on the test data
m3_prob <- predict(logitMod_3, newdata=data_test, type = 'response')
m3_pred <- ifelse(m3_prob > 0.3, 1, 0)

# Compute confusion matrix
cm_3 <- confusionMatrix(factor(m3_pred), factor(data_test$is_churned))
print(cm_3)

### Modelo 4

In [ ]:
# Fit logistic regression model on the training data
logitMod_4 <- glm(is_churned ~  reward_purchase_count_first_7days + 
                  payments_completed_amount_first_7days +
                  first_payment_amount +
                  payments_initiated, 
                  data=data_train, family=binomial(link="logit"))
# summary 
summary(logitMod_4)

# Make predictions on the test data
m4_prob <- predict(logitMod_4, newdata=data_test, type = 'response')
m4_pred <- ifelse(m4_prob > 0.3, 1, 0)

# Compute confusion matrix
cm_4 <- confusionMatrix(factor(m4_pred), factor(data_test$is_churned))
print(cm_4)

- El modelo 1, con las variables predictoras first_payment_amount, payments_initiated, payments_completed, reward_purchase_count_first_7days, payments_completed_amount_first_7days, visits_feature_1, y visits_feature_2,  clasificó correctamente 21149 casos como clase 0 (no churn) y 1589 casos como clase 1 (churn), y clasificó erróneamente 7356 casos como falsos negativos (pronosticados como no churn, pero en realidad sí churn) y 1106 casos como falsos positivos (pronosticados como churn, pero en realidad no churn). La accuracy del modelo es de 0.7288, lo que significa que clasificó correctamente el 72.88% de los casos.

- El modelo 2, con las variables predictoras first_payment_amount y payments_completed, clasificó correctamente 12477 casos como clase 0 (no churn) y 6573 casos como clase 1 (churn), y clasificó erróneamente 2372 casos como falsos negativos (pronosticados como no churn, pero en realidad sí churn) y 9778 casos como falsos positivos (pronosticados como churn, pero en realidad no churn). La accuracy del modelo es de 0.6106, lo que significa que clasificó correctamente el 61% de los casos.

- El modelo 3, con las variables predictoras first_payment_amount, payments_initiated y  reward_purchase_count_first_7days, clasificó correctamente 12126 casos como clase 0 (no churn) y 6570 casos como clase 1 (churn), y clasificó erróneamente 2375 casos como falsos negativos (pronosticados como no churn, pero en realidad sí churn) y 10129 casos como falsos positivos (pronosticados como churn, pero en realidad no churn). La accuracy del modelo es de 0.5992, lo que significa que clasificó correctamente el 59.92% de los casos.

- El modelo 4, con las variables predictoras reward_purchase_count_first_7days, payments_completed_amount_first_7days, first_payment_amount y payments_initiated, clasificó correctamente 11891 casos como clase 0 (no churn) y 6673 casos como clase 1 (churn), y clasificó erróneamente 2272 casos como falsos negativos (pronosticados como no churn, pero en realidad sí churn) y 10364 casos como falsos positivos (pronosticados como churn, pero en realidad no churn). La accuracy del modelo es de 0.595, lo que significa que clasificó correctamente el 59.5% de los casos.

Por tanto, el mejor modelo ha sido el modelo 1, con las variables predictoras first_payment_amount, payments_initiated, payments_completed, reward_purchase_count_first_7days, payments_completed_amount_first_7days, visits_feature_1, y visits_feature_2.

Centrándonos por ende en el modelo 1, observando el resumen (summary) vemos que:

- El resultado muestra el resumen de un modelo de regresión logística con siete variables predictoras: first_payment_amount, payments_initiated, payments_completed, reward_purchase_count_first_7days, payments_completed_amount_first_7days, visits_feature_1, y visits_feature_2. El modelo pretende predecir si un usuario se dará de baja (1) o no (0) en función de estas variables.

- La tabla de coeficientes muestra el efecto estimado de cada variable predictiva sobre la probabilidad de abandono. Un coeficiente negativo indica que un aumento de la variable predictiva se asocia a una disminución de la probabilidad de abandono, mientras que un coeficiente positivo indica lo contrario. Todas las variables predictoras de este modelo tienen coeficientes negativos, lo que sugiere que los valores más altos de estas variables se asocian con menores probabilidades de abandono. Los p-values de todas las variables son inferiores a 0.05, lo que indica que todas las variables predictoras son estadísticamente significativas a la hora de predecir el abandono.

- Los residuos de desviación oscilan entre -1.0206 y 8.4904, con la mayoría de ellos entre -0.9 y 1.3, lo que indica que el modelo es capaz de explicar la mayor parte de la variación de los datos. 

- La diferencia entre las desviaciones (es decir, desviación nula - desviación residual) es la cantidad de desviación explicada por las variables predictoras del modelo. En este caso, las variables predictoras explican una desviación de 4033, lo que indica que el modelo se ajusta bien a los datos.

- El valor AIC  es una medida de la calidad del ajuste del modelo, donde un valor AIC más bajo indica un mejor ajuste del modelo. El valor AIC para este modelo es 83223, que es relativamente bajo, lo que indica que el modelo tiene un buen ajuste.

En conclusión, el modelo de regresión logística muestra que todas las variables predictoras son estadísticamente significativas para predecir el churn. El modelo tiene un buen ajuste, como indican los residuos de desviación y el valor AIC. Los coeficientes negativos de todas las variables predictoras sugieren que los valores más altos de estas variables se asocian con menores probabilidades de abandono. Estos datos podrían ayudar a identificar a los usuarios con riesgo de abandono y a tomar las medidas adecuadas para retenerlos.


<br>
<br>

Observando el resultado de la matriz de confusión (ya comentada anteriormente) y las demás métricas de interés, vemos que:


- La sensibilidad del modelo es de 0.9503, lo que indica que el modelo identifica correctamente al 95.03% de los clientes que abandonaron. La especificidad es de 0.1776, lo que indica que el modelo sólo identifica correctamente al 17.76% de los clientes que no abandonaron. El valor predictivo positivo (VPP) es de 0.7419, lo que indica que, de todos los clientes que el modelo predijo que abandonarían, el 74.19% lo hicieron realmente. El valor predictivo negativo (VPN) es de 0.5896, lo que indica que de todos los clientes que el modelo predijo que no abandonarían, el 58.96% no lo hizo.

- La prevalencia de churns en el conjunto de datos es de 0.7133, y la tasa de detección del modelo es de 0.6779, lo que indica que el modelo detecta el 67.79% de todos los clientes que realmente se dieron de baja. La prevalencia de detección del modelo es de 0.9136, lo que indica que el modelo predice que el 91.36% de los clientes se darán de baja.

- La accuracy equilibrada del modelo es de 0.5640, que es la media de la sensibilidad y la especificidad. En este caso, la clase "positiva" son los clientes que no cambiaron de proveedor y la clase "negativa" son los clientes que cambiaron de proveedor.

En conclusión, las variables predictoras first_payment_amount, payments_initiated, payments_completed, reward_purchase_count_first_7days, payments_completed_amount_first_7days, visits_feature_1, y visits_feature_2 han sido las que mejor resultado han dado. El modelo de regresión logística tiene una accuracy ligeramente superior a la de adivinar la clase mayoritaria, pero la concordancia entre las clases reales y las predichas es sólo ligeramente superior al azar. El modelo tiene una alta sensibilidad para detectar a los clientes que se han dado de baja, pero una baja especificidad para identificar a los clientes que no se han dado de baja. El valor predictivo positivo del modelo es relativamente bueno, pero el valor predictivo negativo es bajo. La prevalencia del churn en el conjunto de datos es alta, y la tasa de detección del modelo es moderada. 

## Conclusiones y vías abiertas

A partir de los resultados del modelo 1 de regresión logística, podemos identificar las variables que más influyen en la pérdida de clientes. Los coeficientes de las variables predictoras sugieren que los siguientes factores tienen un impacto negativo en la pérdida de clientes (es decir, valores más altos de estas variables reducen la probabilidad de pérdida de clientes):

- Mayor importe del primer pago
- Mayor número de pagos iniciados
- Mayor número de pagos completados
- Mayor número de recompensas reclamadas en los primeros 7 días
- Mayor importe de pago completado en los primeros 7 días
- Mayor número de visitas de los usuarios al producto 1
- Mayor número de visitas de los usuarios al producto 2

Por lo tanto, basándose en los resultados, la empresa podría considerar las siguientes recomendaciones para reducir el churn:

- Animar a los usuarios a realizar un primer pago más elevado ofreciéndoles descuentos u ofertas especiales. Esto puede aumentar su compromiso con el servicio y reducir la probabilidad de abandono.

- Aumentar el compromiso con los usuarios animándoles a iniciar más pagos y a completar más pagos. Esto puede lograrse ofreciendo servicios, funciones o incentivos adicionales a los usuarios.

- Ofrecer más recompensas e incentivos durante los primeros 7 días de afiliación para animar a los usuarios a seguir utilizando el servicio.

- Centrarse en mejorar el compromiso de los usuarios con las funciones 1 y 2 del producto, ya que parece tener un fuerte impacto negativo en el churn.

El modelo de regresión logística puede ayudar a la empresa a llegar a estas conclusiones al proporcionar información sobre las variables que más influyen en la pérdida de clientes. Al identificar estas variables, la empresa puede desarrollar estrategias específicas para mejorar estos aspectos del servicio y reducir la pérdida de clientes. Además, el modelo puede utilizarse para predecir la probabilidad de que un usuario abandone el servicio en función de sus características. Esto puede utilizarse para desarrollar estrategias de retención personalizadas y mejorar la eficacia de los esfuerzos de prevención de la empresa.

## Referencias

[1] Chen, L.-P. (2021). Practical Statistics for Data Scientists. Technometrics (Vol. 63, pp. 272–273).

[2] Hastie, T., Tibshirani, R., James, G., & Witten, D. (2021). An introduction to Statistical Learning with Applications in R (2nd Edition). Springer Texts, 102, 618.

[3] Compare multiple sample variances in R. STHDA. Retrieved January 17, 2023, from http://www.sthda.com/english/wiki/compare-multiple-sample-variances-in-r 


[4] https://www.paultwin.com/wp-content/uploads/Lodder_1140873_Paper_Imputation.pdf


[5] https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.496.3324&rep=rep1&type=pdf
